In [3]:
!pip install git+https://github.com/openai/CLIP.git
!pip install pytorch_fid

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-cijcp5ig
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-cijcp5ig
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import random
from diffusers import StableDiffusionPipeline, DDPMScheduler
from PIL import Image
import clip
from pytorch_fid import fid_score
from tqdm import tqdm
import numpy as np

# Import PEFT components for LoRA
from peft import LoraConfig, get_peft_model


In [5]:
# ------------------------------
# Set Up Directories and Device
# ------------------------------
curated_dataset_path = '/kaggle/input/dreambooth-dataset/dataset'  # Reference images per class (curated WikiArt)
generated_base_dir = '/kaggle/working/generated_images_curated'    # Where generated images will be saved

if not os.path.exists(generated_base_dir):
    os.makedirs(generated_base_dir)

device = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------
# Load Stable Diffusion and CLIP Models
# ------------------------------
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
)
pipe = pipe.to(device)

clip_model, clip_preprocess = clip.load("ViT-L/14", device=device)

# ------------------------------
# Functions for Generation and Metrics
# ------------------------------
def generate_images_for_class(prompt, num_images, output_dir):
    """
    Generate images using Stable Diffusion for a given prompt and save them.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    generated_images = []
    for i in range(num_images):
        image = pipe(prompt).images[0]
        image_path = os.path.join(output_dir, f"generated_{i}.png")
        image.save(image_path)
        generated_images.append(image)
    return generated_images

def compute_average_clip_similarity(generated_images, prompt):
    """
    Compute average CLIP cosine similarity between the generated images and the text prompt.
    """
    text_tokens = clip.tokenize([prompt]).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    similarities = []
    for image in generated_images:
        image_input = clip_preprocess(image).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = clip_model.encode_image(image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        similarity = (image_features @ text_features.T).item()
        similarities.append(similarity)
    return np.mean(similarities)

# def compute_average_niqe(generated_images):
#     """
#     Compute average NIQE score for the generated images.
#     (Note: Replace the dummy implementation with an actual NIQE computation.)
#     """
#     def compute_niqe(image_np):
#         # Dummy NIQE computation: Replace this with a proper implementation
#         return np.random.uniform(3.0, 6.0)
    
#     scores = []
#     for image in generated_images:
#         img_np = np.array(image)
#         score = compute_niqe(img_np)
#         scores.append(score)
#     return np.mean(scores)




Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
import os
import shutil
from PIL import Image

def copy_folder(src_folder, dst_folder):
    """
    Copies all files (and optionally subdirectories) from src_folder to dst_folder.
    """
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    for item in os.listdir(src_folder):
        src_path = os.path.join(src_folder, item)
        dst_path = os.path.join(dst_folder, item)
        if os.path.isfile(src_path):
            shutil.copy(src_path, dst_path)
        elif os.path.isdir(src_path):
            shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
    print(f"Copied folder from {src_folder} to {dst_folder}")

def resize_images_in_folder(folder_path, target_size=(299, 299)):
    """
    Resizes all image files in the folder (and subfolders) to the target size.
    """
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as img:
                        # Resize and overwrite the image
                        img_resized = img.resize(target_size, Image.LANCZOS)
                        img_resized.save(file_path)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    print(f"Resized all images in {folder_path} to {target_size}")

In [8]:
dest_folder = '/kaggle/working/resized_dataset'
copy_folder(curated_dataset_path, dest_folder)
resize_images_in_folder(dest_folder, target_size=(512,512))

Copied folder from /kaggle/input/dreambooth-dataset/dataset to /kaggle/working/resized_dataset
Resized all images in /kaggle/working/resized_dataset to (512, 512)


#### Evaluate on pretrained Stable Diffusion

In [8]:
# ------------------------------
# Main Evaluation Loop: Iterate Over Classes
# ------------------------------
num_generated_images = 6  # Number of images to generate per class
results = {}

for class_name in os.listdir(dest_folder):
    class_ref_path = os.path.join(dest_folder, class_name)
    if os.path.isdir(class_ref_path):
        # Create a folder for generated images for this class
        output_dir = os.path.join(generated_base_dir, class_name)
        prompt = f"A photo of {class_name}"
        print(f"\nProcessing class: {class_name}\nPrompt: {prompt}")
        
        # Generate images using the prompt
        generated_images = generate_images_for_class(prompt, num_generated_images, output_dir)

        # for idx, image in enumerate(generated_images):
        #     # Convert the PIL Image to a NumPy array
        #     img_np = np.array(image)
        #     print(f"Image {idx} shape: {img_np.shape}")
        
        # for image in generated_images:
        #     print(image.shape)
        
        # Compute CLIP similarity
        avg_clip_similarity = compute_average_clip_similarity(generated_images, prompt)
        
        # Compute NIQE (dummy implementation here)
        # avg_niqe = compute_average_niqe(generated_images)

        # resize_images_in_folder(class_ref_path, target_size=(512, 512))
        
        # Compute FID: Compare the generated images with the curated reference images for this class
        fid_value = fid_score.calculate_fid_given_paths(
            [class_ref_path, output_dir],
            batch_size=6, device=device, dims=2048
        )
        
        results[class_name] = {
            "avg_clip_similarity": avg_clip_similarity,
            # "avg_niqe": avg_niqe,
            "fid": fid_value
        }
        print(f"Results for {class_name}:")
        print(f"  Average CLIP Similarity: {avg_clip_similarity:.4f}")
        # print(f"  Average NIQE: {avg_niqe:.4f}")
        print(f"  FID: {fid_value:.4f}")

# Optionally: Print summary for all classes
print("\nSummary of Metrics:")
for class_name, metrics in results.items():
    print(f"\nClass: {class_name}")
    print(f"  CLIP Similarity: {metrics['avg_clip_similarity']:.4f}")
    # print(f"  NIQE: {metrics['avg_niqe']:.4f}")
    print(f"  FID: {metrics['fid']:.4f}")


Processing class: teapot
Prompt: A photo of teapot


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  3.81it/s]


Results for cat:
  Average CLIP Similarity: 0.2425
  FID: 169.9074

Summary of Metrics:

Class: teapot
  CLIP Similarity: 0.2833
  FID: 236.2425

Class: dog6
  CLIP Similarity: 0.2180
  FID: 262.6144

Class: pink_sunglasses
  CLIP Similarity: 0.2896
  FID: 274.9720

Class: rc_car
  CLIP Similarity: 0.2877
  FID: 281.9464

Class: candle
  CLIP Similarity: 0.2402
  FID: 377.8771

Class: fancy_boot
  CLIP Similarity: 0.2418
  FID: 387.3131

Class: poop_emoji
  CLIP Similarity: 0.2297
  FID: 477.1788

Class: dog
  CLIP Similarity: 0.2175
  FID: 192.4786

Class: grey_sloth_plushie
  CLIP Similarity: 0.3223
  FID: 199.2792

Class: dog5
  CLIP Similarity: 0.2237
  FID: 276.5315

Class: monster_toy
  CLIP Similarity: 0.2672
  FID: 499.2143

Class: backpack_dog
  CLIP Similarity: 0.2785
  FID: 359.9393

Class: wolf_plushie
  CLIP Similarity: 0.2992
  FID: 203.2172

Class: dog8
  CLIP Similarity: 0.2325
  FID: 259.5497

Class: can
  CLIP Similarity: 0.1858
  FID: 379.6414

Class: dog3
  CLIP Sim

##### Helper functions

In [9]:
##############################################
# 1. Dataset Setup
##############################################
class WikiArtDataset(Dataset):
    def __init__(self, folder, transform):
        self.folder = folder
        self.transform = transform
        self.image_files = [os.path.join(folder, f) for f in os.listdir(folder)
                            if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        path = self.image_files[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transformation for training images: Stable Diffusion typically expects 512x512 images normalized to [-1,1]
train_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),  # scales to [0, 1]
    transforms.Normalize([0.5]*3, [0.5]*3)  # scales to [-1, 1]
])


def compute_clip_similarity(images, prompt, clip_model, clip_preprocess, device):
    """Compute average CLIP cosine similarity between generated images and a prompt."""
    text_tokens = clip.tokenize([prompt]).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarities = []
    for img in images:
        image_input = clip_preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = clip_model.encode_image(image_input)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        sim = (image_features @ text_features.T).item()
        similarities.append(sim)
    return np.mean(similarities)

In [10]:
def is_black_image(image, threshold=5):
    """
    Determines if an image is 'black' by computing the mean pixel value.
    Assumes the image is in RGB format (0-255 scale).
    """
    img_np = np.array(image)
    return np.mean(img_np) < threshold

def filter_black_images(source_folder, target_folder, threshold=5):
    """
    Copies images from source_folder to target_folder only if they are not 'black'.
    """
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(source_folder, filename)
            try:
                image = Image.open(file_path).convert("RGB")
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                continue
            
            if is_black_image(image, threshold):
                print(f"Skipping black image: {filename}")
            else:
                shutil.copy(file_path, os.path.join(target_folder, filename))
                print(f"Copied valid image: {filename}")


In [11]:
def load_images_as_tensor(folder, transform, device):
    """Load all images in a folder as a tensor of shape (N, C, H, W)."""
    images = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('png','jpg','jpeg')):
            path = os.path.join(folder, filename)
            try:
                with Image.open(path).convert("RGB") as img:
                    img_t = transform(img)
                    images.append(img_t)
            except Exception as e:
                print(f"Error loading {path}: {e}")
    if images:
        return torch.stack(images).to(device)
    else:
        return None

#### Stable Diffusion + LoRA Training

In [27]:
device = "cuda" if torch.cuda.is_available() else "cpu"
curated_dataset_path = "/kaggle/working/resized_dataset"  # Folder containing one subfolder per class
output_base_dir = "/kaggle/working/generated_images_lora_finetuned"  # Base directory for generated images
os.makedirs(output_base_dir, exist_ok=True)

# Load CLIP for evaluation
clip_model, clip_preprocess = clip.load("ViT-L/14", device=device)

# Dictionary to store evaluation metrics for each class
metrics_dict = {}

# Get all class folders in the curated dataset
classes = [d for d in os.listdir(curated_dataset_path) if os.path.isdir(os.path.join(curated_dataset_path, d))]
print("Classes found:", classes)

# Iterate over each class (e.g., Romanticism, Impressionism, etc.)
for class_name in classes:
    print(f"\n--- Processing Class: {class_name} ---")
    train_folder = os.path.join(curated_dataset_path, class_name)
    
    # Create dataset and DataLoader for the class
    dataset = WikiArtDataset(train_folder, train_transform)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, drop_last=True)
    
    # Load base Stable Diffusion pipeline
    model_id = "CompVis/stable-diffusion-v1-4"
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to(device)
    unet = pipe.unet
    vae = pipe.vae

    # Load diffusion scheduler (e.g., DDPMScheduler)
    scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")
    
    ##############################################
    # 3a. Apply LoRA Fine-Tuning to the UNet
    ##############################################
    # Configure LoRA for Stable Diffusion's UNet:
    # Targeting cross-attention projection layers (commonly named "to_q", "to_k", and "to_v").
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["to_q", "to_k", "to_v"],
        lora_dropout=0.05,
        bias="none",
        task_type="FEATURE_EXTRACTION",
        fan_in_fan_out=False           # Ensure output dimensions are preserved
    )
    
    unet = get_peft_model(unet, lora_config)

    original_forward = unet.forward
    def patched_forward(*args, **kwargs):
        # Remove "input_ids" from kwargs if present.
        kwargs.pop("input_ids", None)
        kwargs.pop("inputs_embeds", None)
        kwargs.pop("output_attentions", None)
        kwargs.pop("output_hidden_states", None)
        return original_forward(*args, **kwargs)
    unet.forward = patched_forward
    
    # Also patch the underlying base model if it exists.
    if hasattr(unet, "base_model"):
        original_base_forward = unet.base_model.forward
        def patched_base_forward(*args, **kwargs):
            kwargs.pop("input_ids", None)
            kwargs.pop("inputs_embeds", None)
            kwargs.pop("output_attentions", None)
            kwargs.pop("output_hidden_states", None)
            return original_base_forward(*args, **kwargs)
        unet.base_model.forward = patched_base_forward
    
    pipe.unet = unet  # Replace the pipeline's UNet with the LoRA-wrapped version
    
    # Set UNet to training mode (only LoRA parameters should be trainable)
    unet.train()
    optimizer = optim.AdamW(unet.parameters(), lr=1e-5, weight_decay=1e-4)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.9)
    mse_loss = nn.MSELoss()
    
    num_train_steps = 1000  # For demonstration; increase for real experiments
    global_step = 0
    
    # For each class, we use a prompt corresponding to the class style.
    prompt = f"A photo of {class_name}"
    text_input = pipe.tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        text_embeddings = pipe.text_encoder(text_input.input_ids.to(device))[0]
        # print(text_embeddings.shape)
    
    print(f"Fine-tuning for class '{class_name}' with prompt: '{prompt}'")

    num_epochs = 100
    
    # Training loop with diffusion (noise prediction) loss
    for epoch in range(num_epochs):  # One epoch for demonstration
        for images in tqdm(dataloader, desc=f"Training {class_name}"):
            images = images.to(device).half()
            # Encode images to latent space via the VAE
            with torch.no_grad():
                latents = vae.encode(images).latent_dist.sample() * 0.18215
            noise = torch.randn_like(latents).to(device)
            b = latents.shape[0]
            timesteps = torch.randint(0, scheduler.num_train_timesteps, (b,), device=device).long()
            # Add noise to the latents as per the diffusion process
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)

            # Repeat text embeddings for the current batch if necessary
            if text_embeddings.shape[0] == 1 and b > 1:
                text_embeddings = text_embeddings.repeat(b, 1, 1)
                # print("Replicated text embeddings shape:", text_embeddings.shape)

            # print(noisy_latents.shape)
            # Predict the noise using the LoRA-adapted UNet
            noise_pred = unet(sample=noisy_latents, timestep=timesteps, encoder_hidden_states=text_embeddings)[0]

            loss = mse_loss(noise_pred, noise)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(unet.parameters(), max_norm=1.0)
            
            optimizer.step()
            lr_scheduler.step()  # Step the LR scheduler after each optimizer update
            
            global_step += 1
            if global_step % 10 == 0:
                print(f"Step {global_step}: Loss {loss.item():.6f}")
            if global_step >= num_train_steps:
                break
        if global_step >= num_train_steps:
            break
    
    # Save the fine-tuned model for this class
    # class_model_save_path = f"lora_finetuned_{class_name}"
    # pipe.save_pretrained(class_model_save_path)
    # print(f"Saved fine-tuned model for class {class_name} at {class_model_save_path}")
    
    ##############################################
    # 3b. Generate Images with the Fine-Tuned Model
    ##############################################
    gen_output_dir = os.path.join(output_base_dir, class_name)
    os.makedirs(gen_output_dir, exist_ok=True)
    num_generated_images = 6
    generated_images = []
    def inference_forward(sample, timestep, encoder_hidden_states, **kwargs):
        # Remove extra keys that are not expected by UNet2DConditionModel.forward
        for key in ["input_ids", "attention_mask", "inputs_embeds", "output_attentions", "output_hidden_states"]:
            kwargs.pop(key, None)
        # Call the original base model's forward method.
        return unet.base_model.forward(sample=sample, timestep=timestep, encoder_hidden_states=encoder_hidden_states, **kwargs)

    # Replace the pipeline's UNet forward with our inference version.
    pipe.unet.forward = inference_forward
    
    print(f"Generating images for class '{class_name}' with prompt: '{prompt}'")
    for i in range(num_generated_images):
        image = pipe(prompt).images[0]
        save_path = os.path.join(gen_output_dir, f"generated_{i}.png")
        image.save(save_path)
        generated_images.append(image)
    
    # # Resize generated images to 299x299 for FID computation
    # resize_images_in_folder(gen_output_dir, target_size=(299, 299))
    # resize_images_in_folder(train_folder, target_size=(512, 512))
    
    ##############################################
    # 3c. Compute Evaluation Metrics
    ##############################################
    
    # Compute FID between training images (reference) and generated images
    gen_filtered_output_dir = os.path.join(output_base_dir, class_name + "_filtered")
    os.makedirs(gen_filtered_output_dir, exist_ok=True)
    filter_black_images(gen_output_dir, gen_filtered_output_dir)
    
    fid_value = fid_score.calculate_fid_given_paths(
        [train_folder, gen_output_dir],
        batch_size=2, device=device, dims=2048
    )

    transform_kid = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
    ])
    
    # Load reference images and generated images as tensors
    reference_tensors = load_images_as_tensor(train_folder, transform_kid, device)
    generated_tensors = load_images_as_tensor(gen_filtered_output_dir, transform_kid, device)
    
 
    filtered_generated_images = []
    for img in generated_images:
        if is_black_image(img):
            print("Skipping NSFW/black image")
            continue
        filtered_generated_images.append(img)
    
    # Compute CLIP similarity between generated images and the prompt
    clip_sim = compute_clip_similarity(generated_images, prompt, clip_model, clip_preprocess, device)
    
    print(f"Class {class_name} - FID: {fid_value:.4f}, CLIP Similarity: {clip_sim:.4f}")
    metrics_dict[class_name] = {"FID": fid_value, "CLIP_Similarity": clip_sim}

##############################################
# 4. Summary of Metrics for All Classes
##############################################
print("\nSummary of Evaluation Metrics for All Classes:")
for cls, metrics in metrics_dict.items():
    print(f"Class: {cls} -> FID: {metrics['FID']:.4f}, CLIP Similarity: {metrics['CLIP_Similarity']:.4f}")

Classes found: ['poop_emoji', 'duck_toy', 'pink_sunglasses', 'shiny_sneaker', 'vase', 'colorful_sneaker', 'dog3', 'dog', 'dog6', 'clock', 'berry_bowl', 'can', 'dog2', 'red_cartoon', 'wolf_plushie', 'robot_toy', 'candle', 'dog7', 'rc_car', 'dog8', 'cat2', 'teapot', 'backpack', 'backpack_dog', 'monster_toy', 'grey_sloth_plushie', 'cat', 'fancy_boot', 'bear_plushie', 'dog5']

--- Processing Class: poop_emoji ---


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Fine-tuning for class 'poop_emoji' with prompt: 'A photo of poop_emoji'


Training poop_emoji: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


Step 10: Loss 0.299805


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Copied valid image: generated_2.png
Copied valid image: generated_0.png
Copied valid image: generated_4.png
Copied valid image: generated_5.png
Copied valid image: generated_1.png
Copied valid image: generated_3.png


100%|██████████| 3/3 [00:00<00:00, 10.63it/s]


Class dog5 - FID: 238.6892, CLIP Similarity: 0.2264

Summary of Evaluation Metrics for All Classes:
Class: poop_emoji -> FID: 578.4787, CLIP Similarity: 0.2319
Class: duck_toy -> FID: 288.9263, CLIP Similarity: 0.2783
Class: pink_sunglasses -> FID: 216.5150, CLIP Similarity: 0.2965
Class: shiny_sneaker -> FID: 222.5358, CLIP Similarity: 0.2659
Class: vase -> FID: 348.7170, CLIP Similarity: 0.2553
Class: colorful_sneaker -> FID: 243.8247, CLIP Similarity: 0.2574
Class: dog3 -> FID: 273.7140, CLIP Similarity: 0.2299
Class: dog -> FID: 210.2617, CLIP Similarity: 0.2197
Class: dog6 -> FID: 247.5963, CLIP Similarity: 0.2182
Class: clock -> FID: 302.0461, CLIP Similarity: 0.2296
Class: berry_bowl -> FID: 387.9627, CLIP Similarity: 0.2514
Class: can -> FID: 440.5197, CLIP Similarity: 0.2162
Class: dog2 -> FID: 267.8411, CLIP Similarity: 0.2251
Class: red_cartoon -> FID: 325.1048, CLIP Similarity: 0.2262
Class: wolf_plushie -> FID: 214.9455, CLIP Similarity: 0.2968
Class: robot_toy -> FID: 298

In [ ]:
import os
import zipfile
from IPython.display import FileLink, display

def zip_filtered_folders(base_dir, target_suffix="filtered"):
    """
    Finds subdirectories in base_dir whose names end with target_suffix,
    creates a zip archive for each, and returns a list of paths to the zip files.
    """
    zip_files = []
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path) and folder.endswith(target_suffix):
            zip_path = os.path.join(base_dir, f"{folder}.zip")
            with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
                for root, dirs, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        # Create an archive name relative to base_dir
                        arcname = os.path.relpath(file_path, base_dir)
                        zipf.write(file_path, arcname)
            print(f"Zipped folder {folder} to {zip_path}")
            zip_files.append(zip_path)
    return zip_files

# Set your base directory (adjust as needed)
base_directory = "/kaggle/working/generated_images_lora_finetuned"  # Replace with your directory path

# Zip folders ending with "filtered"
zip_file_list = zip_filtered_folders(base_directory, target_suffix="filtered")

# Display download links for each zip file
print("\nDownload Links:")
for zfile in zip_file_list:
    display(FileLink(zfile))

